In [0]:
from pyspark.sql.functions import col, sum, round, desc

Faturamento Real: Qual o faturamento total por categoria de produto (considerando apenas pedidos com pagamento "Aprovado")?

Top Clientes: Quais são os 5 clientes que mais gastaram na loja?

Qualidade vs Vendas: Produtos com avaliações ruins (abaixo de 3) estão tendo muitas devoluções ou baixo volume de vendas?

In [0]:
df_orders = spark.read.table("workspace.datamart.silver_orders")
df_payments = spark.read.table("workspace.datamart.silver_payments")
df_products = spark.read.table("workspace.datamart.silver_products")
df_customers = spark.read.table("workspace.datamart.silver_customers")

In [0]:
#print("df_orders")
#display(df_orders)
#print("df_payments")
#display(df_payments)
#print("df_products")
#display(df_products)
#print("df_customers")
#display(df_customers)

In [0]:
#  Criando tabela Gold - Cruzamento e Agregação

df_faturamento = (
    df_orders
    .join(df_payments, "order_id", how = "left")                        # Junta com pagamentos usando a coluna em comum (order_id)
    .filter(col("status").ilike("%aprovado"))                           # Filtra apenas os aprovados (usando ilike para ignorar maiúsculas/minúsculas por segurança)
    
    .join(df_products, "product_id",how = "left")                       # Junta com produtos para trazer o nome da categoria
    .withColumn("valor_total_item", col("quantity")*col("unit_price"))  # Cria a coluna de valor total daquela linha (Quantidade * Preço Unitário)
    
    .groupBy("category", "product_name")                                # Agrupa por categoria e demais campos e soma o valor total
    .agg(
        round(sum("valor_total_item"),2).alias("faturamento_aprovado")
    )
    .orderBy(desc("faturamento_aprovado"))                              # Ordena do maior faturamento para o menor
)

#print("🏆 Camada Gold: Faturamento por Categoria")
#display(df_faturamento)

In [0]:
# 2. Lista de clientes que mais compraram (gastaram)
df_clientes = (
    df_orders
    .join(df_customers, "customer_id", "left")
    .withColumn("gasto_item", col("unit_price")* col("quantity"))
    .join(df_payments, "order_id", "left")
    .filter(col("status").ilike("%aprovado"))

    .groupBy("customer_id", "name", "email")
    .agg(round(sum("gasto_item"),2).alias("gasto_total"))
    .orderBy(desc("gasto_total"))
    
)
#print("🏆 Camada Gold: gasto por cliente")
#display(df_clientes)

In [0]:
# Salvando em Tabelas

df_faturamento.write.mode("overwrite").format("delta").saveAsTable("Workspace.Datamart.gold_faturamento")
df_clientes.write.mode("overwrite").format("delta").saveAsTable("Workspace.Datamart.gold_clientes")